# Práctica 3 - CNN para clasificar imágenes de frutas - Lab 1
## Preparación de entorno
#### Instalar las librerías

In [ ]:
#%pip install numpy
#%pip install pandas
#%pip install matplotlib
#%pip install pytorch

#### Importar librerías de código

## CNN SetUp
* **Red convolucional secuencial (CNN secuencial)** $\rightarrow$ Tipo de red neuronal convolucional diseñada utilizando un modelo secuenncial (una pila lineal de capas donde cada capa recibe la salida de la capa anterior como entrada).
  * **Convolución** $\rightarrow$ Operación matemática que combina dos funciones para producir una tercera función. En el contexto de las CNN, se utiliza para extraer características de las imágenes.
  * **Capas** $\rightarrow$ Capas de convolución, activación, pooling y completamente conectadas (fully connected).
  * Recibe **varias entradas** (como señales), cada una con un peso que indica su importancia.

* Componentes principales de una CNN secuencial:
  * **Capas convolucionales** $\rightarrow$ Aplican filtros para extraer características relevantes de las imágenes, como bordes, texturas o patrones. El tamaño y número de filtros son hiperparámetros que definimos nosotros.
  * **Funciones de activación** $\rightarrow$ Generalmente, se utiliza *ReLU* para introducir no linealidad en las capas intermedias, mientras que *Softmax* se utiliza en la capa de salida para problemas de clasificación multiclase.
  * **Capas de pooling** $\rightarrow$ Reducen las dimensiones espaciales (alto y ancho) de las características manteniendo las más relevantes. Esto disminuye el coste computacional y nos ayuda a prevenir el sobreajuste.
  * **Capas completamente conectadas** $\rightarrow$ Al final, las características que hemos extraído las aplanamos y las pasamos a una o más capas densas (completamente conectadas) para hacer la clasificación final.
  * **Optimizador y fución de pérdida** $\rightarrow$ Utilizamos *RMSprop* como optimizador y *categorical_crossentropy* como función de pérdida para problemas de clasificación multiclase. Estos son hiperparámetros que podemos ajustar según nuestras necesidades.

<img src="./media/Estructura cnn.jpg" width="70%" style="display: block; margin: 0 auto;"/>

<img src="./media/Arquitectura cnn.jpg" width="70%" style="display: block; margin: 0 auto; padding-top: 15px;"/>

* Aplicaciones de las redes convolucionales:
  * **Clasificación de imágenes** $\rightarrow$ Identificar objetos en imágenes, como en nuestro caso (clasificar frutas).
  * **Detección de objetos** $\rightarrow$ Localizar y clasificar múltiples objetos en una imagen.
  * **Segmentación de imágenes** $\rightarrow$ Dividir una imagen en regiones significativas, como identificar diferentes partes de una imagen médica.
  * **Reconocimiento facial** $\rightarrow$ Identificar y verificar caras en imágenes o videos.
  * **Procesamiento de video** $\rightarrow$ Analizar secuencias temporales como en coches autónomos o vigilancia.
  * **Generación de imágenes** $\rightarrow$ Crear imágenes nuevas a partir de datos existentes, como en el caso de *GANs* (Generative Adversarial Networks).
  * **Reconocimietnto de texto** $\rightarrow$ Extraer texto de imágenes (OCR).

**Referencias**
* [Convolutional neural network - ScienceDirect](https://www.sciencedirect.com/topics/computer-science/convolutional-neural-network).